In [1]:
import pandas as pd 
import numpy as np
from scipy.stats import gmean
import itertools
import math
from process_collisions_summary import *

Raw data has 17009 samples
N=2^16 has 2859 samples
---------> 279 samples, nwords=2^4, diff=7
---------> 318 samples, nwords=2^5, diff=6
---------> 336 samples, nwords=2^6, diff=6
---------> 376 samples, nwords=2^7, diff=5
---------> 380 samples, nwords=2^8, diff=5
---------> 378 samples, nwords=2^9, diff=4
---------> 391 samples, nwords=2^10, diff=4
---------> 401 samples, nwords=2^11, diff=3
N=2^18 has 4989 samples
---------> 414 samples, nwords=2^4, diff=8
---------> 492 samples, nwords=2^5, diff=7
---------> 498 samples, nwords=2^6, diff=7
---------> 613 samples, nwords=2^7, diff=6
---------> 580 samples, nwords=2^8, diff=6
---------> 583 samples, nwords=2^9, diff=5
---------> 602 samples, nwords=2^10, diff=5
---------> 630 samples, nwords=2^11, diff=4
---------> 577 samples, nwords=2^12, diff=4
N=2^20 has 5826 samples
---------> 213 samples, nwords=2^4, diff=9
---------> 392 samples, nwords=2^5, diff=8
---------> 393 samples, nwords=2^6, diff=8
---------> 519 samples, nwords=2^7, 

___

# Simulations Summary
**Please note that for some sizes the number of experiments is small**

In [2]:
nsamples_df = pd.DataFrame(nsamples).set_index("nwords").transpose()
print(f"number of samples processed = {nsamples_df.sum().sum()}")
nsamples_df = nsamples_df.rename_axis("nwords").rename_axis("N")


nsamples_df = nsamples_df.style.set_caption("Number of Samples Collected")
nsamples_df

number of samples processed = 16999


nwords,2^4,2^5,2^6,2^7,2^8,2^9,2^10,2^11,2^12,2^13,2^14,2^15
N,,,,,,,,,,,,
2^16,279,318,336,376,380,378,391,401,-1,-1,-1,-1
2^18,414,492,498,613,580,583,602,630,577,-1,-1,-1
2^20,213,392,393,519,531,722,702,762,811,781,-1,-1
2^22,21,48,34,108,76,207,273,351,426,419,494,-1
2^24,2,2,3,6,15,15,36,65,104,177,173,280


In [3]:
factor_n3w_df = pd.DataFrame(factor_n3w).set_index("nwords").transpose()
factor_n3w_df = factor_n3w_df.rename_axis("nwords").rename_axis("N")
factor_n3w_df = factor_n3w_df.style.set_caption("factor of sqrt(N^3/nwords)")
factor_n3w_df

nwords,2^4,2^5,2^6,2^7,2^8,2^9,2^10,2^11,2^12,2^13,2^14,2^15
N,,,,,,,,,,,,
2^16,0.847808,2.163837,4.118726,2.791029,3.077228,2.092568,2.539370,1.545029,-1.000000,-1.000000,-1.000000,-1.000000
2^18,0.695921,2.119274,3.544303,4.752419,3.917689,2.900965,3.068458,2.018334,2.400777,-1.000000,-1.000000,-1.000000
2^20,0.262942,1.141138,1.543996,4.395545,4.370746,4.471315,4.564345,3.058567,3.481021,2.117325,-1.000000,-1.000000
2^22,0.042157,0.226547,0.264657,1.090551,1.667159,2.887740,3.579167,3.081696,3.711106,2.314587,2.938625,-1.000000
2^24,0.018636,0.000684,0.068779,0.123851,0.225109,0.409037,0.832824,1.263011,1.238537,1.481185,1.756441,1.743433


In [4]:
nfunction_updates_df = pd.DataFrame(nfunction_updates).set_index("nwords").transpose()
nfunction_updates_df = nfunction_updates_df.rename_axis("nwords").rename_axis("N")
nfunction_updates_df = nfunction_updates_df.style.set_caption("Number of Function Updates")
nfunction_updates_df

nwords,2^4,2^5,2^6,2^7,2^8,2^9,2^10,2^11,2^12,2^13,2^14,2^15
N,,,,,,,,,,,,
2^16,2^10.36,2^10.65,2^10.05,2^8.75,2^7.53,2^6.41,2^5.05,2^3.88,-1,-1,-1,-1
2^18,2^12.02,2^12.52,2^11.83,2^11.53,2^9.85,2^8.84,2^7.37,2^6.27,2^5.04,-1,-1,-1
2^20,2^12.41,2^13.49,2^12.46,2^13.36,2^11.90,2^11.38,2^9.92,2^8.86,2^7.49,2^6.23,-1,-1
2^22,2^11.78,2^12.98,2^11.96,2^13.19,2^12.17,2^12.65,2^11.34,2^10.88,2^9.45,2^8.47,2^7.24,-1
2^24,2^11.89,2^7.43,2^11.53,2^12.18,2^11.38,2^11.82,2^11.03,2^11.11,2^9.89,2^9.68,2^8.37,2^7.90


In [5]:
avgs_df = pd.DataFrame(avgs).set_index("nwords").transpose()
avgs_df = avgs_df.rename_axis("nwords").rename_axis("N")
avgs_df = avgs_df.style.set_caption("Geometric Mean of #points generated")
avgs_df

nwords,2^4,2^5,2^6,2^7,2^8,2^9,2^10,2^11,2^12,2^13,2^14,2^15
N,,,,,,,,,,,,
2^16,2^21.76,2^22.61,2^23.04,2^21.98,2^21.62,2^20.57,2^20.34,2^19.13,-1,-1,-1,-1
2^18,2^24.48,2^25.58,2^25.83,2^25.75,2^24.97,2^24.04,2^23.62,2^22.51,2^22.26,-1,-1,-1
2^20,2^26.07,2^27.69,2^27.63,2^28.64,2^28.13,2^27.66,2^27.19,2^26.11,2^25.80,2^24.58,-1,-1
2^22,2^26.43,2^28.36,2^28.08,2^29.63,2^29.74,2^30.03,2^29.84,2^29.12,2^28.89,2^27.71,2^27.56,-1
2^24,2^28.25,2^22.99,2^29.14,2^29.49,2^29.85,2^30.21,2^30.74,2^30.84,2^30.31,2^30.07,2^29.81,2^29.30


In [6]:
stds_df = pd.DataFrame(stds)
stds_df = stds_df.set_index("nwords")
stds_df = stds_df.transpose()
stds_df = stds_df.rename_axis("nwords").rename_axis("N")
stds_df = stds_df.style.set_caption("standard variation of log2(#points)")
stds_df

nwords,2^4,2^5,2^6,2^7,2^8,2^9,2^10,2^11,2^12,2^13,2^14,2^15
N,,,,,,,,,,,,
2^16,2.029880,1.966434,1.953175,1.915217,2.100229,2.209735,1.970902,2.092869,-1.000000,-1.000000,-1.000000,-1.000000
2^18,1.938118,1.855880,1.931764,1.961124,2.032700,1.984213,2.023557,2.106842,2.132774,-1.000000,-1.000000,-1.000000
2^20,1.666289,1.804418,1.870437,1.894095,2.037571,1.926532,1.972193,2.003128,1.994903,1.927126,-1.000000,-1.000000
2^22,1.667939,1.503589,1.944907,1.900061,1.572249,2.007746,1.792604,2.059488,1.852708,2.098598,2.081749,-1.000000
2^24,0.542052,3.297016,1.240669,2.222057,1.797574,2.453206,1.569412,1.572717,2.011050,1.994296,1.884626,1.927234
